Варианты того, как можно решить эту задачу:

- Написать правила с помощью spacy matcher (не очень мне нравится, но работает для английского), используя синтаксические шаблоны (я заметила, что чаще всего в текстах продукты упоминаются в предложениях типа "I used PRODUCT for..." или "This PRODUCT is very..."). Плюсы: достаточно легко и быстро, найдутся как упоминания названий товаров (I used Dreamweaver for...) так и их дескрипторы (This visual studio is really great...), + из некоторых таких паттернов можно сразу извлечь и нужную информацию об оценке продукта. Минусы: все паттерны мы не охватим все равно, скорее всего выделим что-то лишнее


- Использовать какую-нибудь модель для извлечения ключевых слов (например, tf-idf), полагаясь на то, что категории товаров разнородны, а корпус большой, и для каждого текста отзыва название товара/его категория будут ключевыми. Плюсы: опять же выделятся и сами названия продуктов, и их более общие названия (в теории). Минусы: лучше сработает для категории товаров с более разнообразными товарами (для Software посложнее, там синонимов немного). Скорее всего выделится что-то лишнее (частотные слова - не названия продукта, но связанные с ним названия его фичей, например, хотя это тоже может быть полезно, если задача, например, собрать общие оценки пользователей не только по продукту, но и по его отдельным аспектам), нужна дополнительная фильтрация


- Собрать список дескрипторов: руками + воспользоваться каким-нибудь тезаурусом типа Ворднет и собрать все гипонимы данной категории (хорошо подходит для конкретных категорий с разными видами продуктов: например, для косметики) и извлекать их из текста. Плюсы: скорее всего, охватим почти все возможные дексрипторы. Минусы: не охватим сами названия брендов (только если они будут в тексте идти вместе - Chanel Perfume)


- Использовать готовую NER модель. Плюсы: ничего делать не нужно, инструмент готов. Найдем все упоминания названий товаров. Минусы: модель может совершать ошибки, когда названия сущностей омонимичны сущностям другого рода. Не найдем случаи, когда товар назван просто "телефон" и тд.


- Комбинация из двух последних способов - объединить предсказания этих двух подходов. Плюсы: соберем и все дескриптивные упоминания товаров, и все их названия. Минусы: не для всех типов товаров подходит подход.

Я выбрала первы подход из-за специфики выбранных данных - у товаров этой категории не так много дескрипторов и в целом они не очень разнородны, беглый взгляд на данные как будто подсказывает, что нужно скорее обратить внимание на синтаксис, чем на лексику

In [179]:
from collections import Counter
import pandas as pd
import pickle
from tqdm.auto import tqdm

import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans

import string

import nltk
from nltk.tokenize import word_tokenize, MWETokenizer
from nltk.collocations import BigramCollocationFinder

In [122]:
with open('Software_5.json') as file:
    raw_data = file.read()
    
nlp = spacy.load("en_core_web_sm")

In [123]:
reviews = raw_data.split('\n')[:-1]
data = pd.DataFrame()

for review in tqdm(reviews):
    data = data.append(eval(review), ignore_index=True)

In [124]:
data.dropna(subset=['reviewText'], inplace=True)
data.tail(5)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
12800,4.0,False,"07 16, 2016",A1E50L7PCVXLN4,B01FFVDY9M,{'Platform:': ' Key Card'},Colinda,When I ordered this it was listed as Photo Edi...,File Management Software with Basic Editing Ca...,1.468627e+09,NaN,NaN
12801,3.0,False,"06 17, 2017",AVU1ILDDYW301,B01HAP3NUG,NaN,G. Hearn,This software has SO much going on. Theres a ...,"Might not be for the ""novice""",1.497658e+09,NaN,NaN
12802,4.0,False,"01 24, 2017",A2LW5AL0KQ9P1M,B01HAP3NUG,NaN,Dr. E,I have used both more complex and less complex...,"Great, Inexpensive Software for Those Who Have...",1.485216e+09,NaN,NaN
12803,3.0,False,"06 14, 2018",AZ515FFZ7I2P7,B01HAP47PQ,{'Platform:': ' PC Disc'},Jerry Jackson Jr.,Pinnacle Studio 20 Ultimate is a perfectly ser...,Gets the job done ... but not as easy as it sh...,1.528934e+09,NaN,NaN
12804,4.0,False,"04 16, 2018",A2WPL6Y08K6ZQH,B01HAP47PQ,{'Platform:': ' PC Disc'},Narut Ujnat,A program that is fairly easy to use and provi...,Good overall program.,1.523837e+09,NaN,NaN


In [125]:
reviews_texts = data.reviewText.values
summ_texts = data.summary.values

Для того, чтобы лучше понять, какие правила писать, посчитаем самые частотные глаголы в текстах ревью и самые частотные существительные в текстах саммари

In [7]:
verbs_freq = Counter()

lemm_reviews_texts = []
for review in tqdm(reviews_texts):
    doc = nlp(review)
    
    lemm_review = []
    for token in doc:
        lemma = token.lemma_
        lemm_review.append(lemma) 
        if token.pos_ == 'VERB':
            verbs_freq[lemma] += 1
            
    lemm_reviews_texts.append(' '.join(lemm_review))

In [11]:
nouns_freq = Counter()

lemm_summ_texts = []
for summary in tqdm(summ_texts):
    if not pd.isna(summary):
        doc = nlp(summary)

        lemm_summary = []
        for token in doc:
            lemma = token.lemma_
            lemm_summary.append(lemma) 
            if token.pos_ == 'NOUN':
                nouns_freq[lemma] += 1

        lemm_summ_texts.append(' '.join(lemm_summary))
    else:
        lemm_summ_texts.append([])

In [12]:
nouns_freq.most_common(50)

[('star', 1336),
 ('software', 625),
 ('product', 609),
 ('program', 315),
 ('version', 298),
 ('year', 244),
 ('price', 220),
 ('feature', 207),
 ('time', 190),
 ('work', 186),
 ('computer', 172),
 ('user', 153),
 ('tax', 149),
 ('problem', 143),
 ('upgrade', 134),
 ('security', 134),
 ('video', 130),
 ('pc', 129),
 ('protection', 126),
 ('money', 118),
 ('way', 113),
 ('lot', 108),
 ('editing', 102),
 ('tool', 99),
 ('office', 94),
 ('update', 91),
 ('support', 90),
 ('business', 86),
 ('value', 85),
 ('alternative', 81),
 ('device', 79),
 ('home', 78),
 ('job', 77),
 ('improvement', 75),
 ('issue', 73),
 ('thing', 70),
 ('fun', 68),
 ('review', 68),
 ('taxis', 68),
 ('internet', 67),
 ('file', 64),
 ('photo', 60),
 ('package', 59),
 ('bit', 57),
 ('need', 55),
 ('system', 55),
 ('install', 55),
 ('interface', 54),
 ('suite', 53),
 ('switch', 51)]

In [209]:
verbs_freq.most_common(50)

[('use', 16696),
 ('can', 11897),
 ('will', 9406),
 ('work', 6888),
 ('make', 5870),
 ('would', 5325),
 ('go', 5169),
 ('find', 5069),
 ('need', 5030),
 ('get', 4512),
 ('want', 4468),
 ('run', 4321),
 ('take', 4132),
 ('try', 3690),
 ('instal', 3000),
 ('give', 2967),
 ('say', 2918),
 ('come', 2907),
 ('buy', 2906),
 ('may', 2895),
 ('think', 2860),
 ('have', 2799),
 ('could', 2770),
 ('see', 2749),
 ('like', 2712),
 ('include', 2710),
 ('look', 2705),
 ('know', 2576),
 ('seem', 2522),
 ('do', 2415),
 ('learn', 2360),
 ('install', 2283),
 ('start', 2153),
 ('recommend', 2026),
 ('create', 2020),
 ('should', 1900),
 ('keep', 1888),
 ('add', 1877),
 ('upgrade', 1801),
 ('allow', 1656),
 ('download', 1578),
 ('offer', 1442),
 ('pay', 1374),
 ('save', 1355),
 ('set', 1319),
 ('link', 1297),
 ('provide', 1285),
 ('update', 1283),
 ('let', 1265),
 ('help', 1248)]

Определяем некоторое кооличество правил с помощью spacy matcher, которые помогут нам достать как общие названия типов продуктов, так и их бренды

In [127]:
matcher = Matcher(nlp.vocab)

#  I used Microsoft Office, I liked this product
pattern1 = [{"LEMMA": {"IN": ["use", "like", "instal"]}}, 
            {"lower": "this", "OP": "*"}, 
            {"POS": "PROPN", "OP": "+"}]

#  This player is amazing, This swith has useful features...
pattern2 = [{"lower": "this"}, 
            {"POS": {"IN": ["PROPN", "NOUN"]}, "OP": "+"}, 
            {"LEMMA": {"IN": ["be", "have"]}}, 
            {"POS": "ADJ", "OP": "*"}]


#  Nancy Drew game,  Verbarrator software etc.
pattern3 = [{"POS": "PROPN"}, 
            {"POS": "PROPN", "OP": "*"}, 
            {"lower": {"IN":["program", "software", "player", "package", "tool", "game"]}}]

matcher.add("verb_pattern", [pattern1])
matcher.add("this_pattern", [pattern2])
matcher.add("descriptor_pattern", [pattern3])

Делаем парсинг и выделяем сами строки с упоминаниями товаров

In [89]:
def get_spans(text):
    spans = []
    
    doc = nlp(text)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id] 
        
        spans.append(doc[start:end])
          
    filt_spans = filter_spans(spans)
    
    return filt_spans

In [111]:
def extract_products(match):
    tokens = [token.text.lower() for token in match]
    
    if tokens[0] in ["use", "like", "instal"]:
        product = ' '.join(tokens[1:])
        
    elif 'this' in tokens:
        this_ind = tokens.index('this')
        
        if 'be' in tokens:
            verb_ind = tokens.index('be')
        elif 'have' in tokens:
            verb_ind = tokens.index('have')
        else:
            print(tokens)
            return None
            
        product = ' '.join(tokens[this_ind+1 : verb_ind])
        
    elif tokens[-1] in ["program", "software", "player", "package", "tool", "game"]:
        product = ' '.join(tokens)
        
    else:
        print(tokens)
        return None
    
    return product

In [117]:
def get_products_mentions(text):
    spans = get_spans(text)
    
    all_prodnames = []
    
    for ind, span in enumerate(spans):
        mention = extract_products(span)
        if mention:
            all_prodnames.append(mention)
    return all_prodnames

In [118]:
extracted_all = []

for text in tqdm(lemm_reviews_texts):
    extracted = get_products_mentions(text)
    extracted_all.append(extracted)

['installed', 'microsoft', 'visual', 'studio']
['used', 'snoop']
['this', 'software.i', 'am']
['this', 'program']
['used', 'os']



В распечатанных примерах что-то пошло не так из-за ошибок лемматизации

In [131]:
extracted_all = sum(extracted_all, [])

In [210]:
extracted_all[:50]

['dreamweaver',
 'course',
 'courseware',
 'course',
 'flash files',
 'flash video',
 'ap',
 'spry',
 'dw',
 'dreamweaver',
 'tutorial',
 'dreamweaver',
 'spry',
 'dreamweaver',
 'front page',
 'adobe lightroom',
 'adobe bridge',
 'lightroom',
 'video series',
 'photoshop',
 'flash cs5',
 'ms windows',
 'flash cs5',
 'flash tool',
 'software',
 'office',
 'year',
 'office products',
 'ms office',
 'ms office software',
 'package',
 'office',
 'iti',
 'microsoft package',
 'office program',
 'office version',
 'outlook',
 'office software',
 'review',
 'version',
 'outlook',
 'google drive',
 'microsoft office',
 'thing',
 'gmail',
 'outlook',
 'office',
 'office',
 'office',
 'office']

Выделенные сущности выглядят неплохо. Токенизируем тексты по-новому (с учетом выделенных биграмм), выделяем все биграммы из тексты и оставляем только те, в которых есть наши сущности + не-пунктуация

In [160]:
#  токенизатор, который не будет разделять сущности из двух слов (при подсчете биграм будем считать их за один токен)
tokenizer_mwe = MWETokenizer(separator=" ")

for entity in extracted_all:
    tokenizer_mwe.add_mwe(tuple(entity.split()))

In [195]:
bigrams_counter = Counter()
tokenized_texts = []

for text in tqdm(reviews_texts):
    tokens = tokenizer_mwe.tokenize(word_tokenize(text.lower()))
    tokenized_texts.append(tokens)
    
    bigrams_text = list(nltk.bigrams(tokens))
    filtered_bigrams = []
    
    for bigram in bigrams_text:
        if bigram[0] not in string.punctuation and bigram[1] not in string.punctuation:
            
            if bigram[0] in extracted_all:
                filtered_bigrams.append(bigram)
                
            elif bigram[1] in extracted_all:
                filtered_bigrams.append(bigram)
   
    bigrams_counter.update(filtered_bigrams)

In [196]:
all_bigrams = bigrams_counter.keys()
len(all_bigrams)

92505

Считаем PMI для всех биграмм

In [180]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(tokenized_texts)

In [211]:
pmi_scores = []
loglike_scores = []
t_scores = []

for bigram in tqdm(all_bigrams):
    pmi = finder.score_ngram(bigram_measures.pmi, bigram[0], bigram[1])
    pmi_scores.append((bigram, pmi))
    
    loglike = finder.score_ngram(bigram_measures.likelihood_ratio, bigram[0], bigram[1])
    loglike_scores.append((bigram, loglike))
    
    tscore = finder.score_ngram(bigram_measures.student_t, bigram[0], bigram[1])
    t_scores.append((bigram, tscore))

Сохраняем все в отдельный датафрейм, группируем по сущности и оставляем в каждой группе только 5 первых результатов. Сущностей выделено много, поэтому сохраняем данные в отдельный csv-файл (лежит в репозитории), чтобы анализировать результаты

In [212]:
def get_item_group(bigram):
    if bigram[0] in extracted_all:
        return bigram[0]
    elif bigram[1] in extracted_all:
        return bigram[1]

res_df = pd.DataFrame()
res_df['bigram'] = [b[0] for b in pmi_scores]
res_df['pmi'] = [b[1] for b in pmi_scores]
res_df['loglike'] = [b[1] for b in loglike_scores]
res_df['tscore'] = [b[1] for b in t_scores]
res_df['item_group'] = res_df['bigram'].apply(get_item_group)

In [215]:
new = res_df[['item_group', 'bigram', 'pmi']].groupby('item_group').apply(lambda x: x.sort_values('pmi', ascending = False)).reset_index(drop=True)
res_pmi = new.groupby('item_group').head(5)

new = res_df[['item_group', 'bigram', 'loglike']].groupby('item_group').apply(lambda x: x.sort_values('loglike', ascending = False)).reset_index(drop=True)
res_loglike = new.groupby('item_group').head(5)

new = res_df[['item_group', 'bigram', 'tscore']].groupby('item_group').apply(lambda x: x.sort_values('tscore', ascending = False)).reset_index(drop=True)
res_tscore = new.groupby('item_group').head(5)

In [216]:
res_pmi.to_csv('pmi_result.csv')
res_loglike.to_csv('loglike_result.csv')
res_tscore.to_csv('tscore_result.csv')

T-score плохо справляется с задачей и больше всего выделяет сочетание сущностей с частотными стоп-словами, чуть лучше, но все еще не очень хорошие результаты у loglikelihood, PMI справляется лучше остальных и для более частотных сущностей иногда выделяется что-то полезное (('functional', 'accounting software'), ('wonderful', 'adobe photoelements program'))

Для решения задачи разрешения синонимии типа "watch", "smartwatch" я бы считала jaccard_similarity_score или что-то похоже дляя выделенных сущностей в рамках одного текста, и если они пересекают определенный порог этой близости, считала бы их относящимися к одной сущности (но реализовать это не успела) 